In [1]:
#https://medium.com/@bobhaffner/gist-to-medium-test-db3d51b8ba7b

In [3]:
import pandas as pd
import dask.dataframe as dd
import fastparquet

In [7]:
csv_file = 'yellow_tripdata_2017-01.csv'
csv_file_on = 'yellow_tripdata_2017-01'

In [4]:
# https://data.world/nyc-taxi-limo/yellow-tripdata-january-2017

%time pd_frame = pd.read_csv('yellow_tripdata_2017-01.csv')
print ('{:,} rows'.format(len(pd_frame)))

Wall time: 20.5 s
9,710,124 rows


In [21]:
pd_frame.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2017-01-09 11:13:28,2017-01-09 11:25:45,1,3.30,1,N,263,161,1,12.5,0.0,0.5,2.00,0.0,0.3,15.30
1,1,2017-01-09 11:32:27,2017-01-09 11:36:01,1,0.90,1,N,186,234,1,5.0,0.0,0.5,1.45,0.0,0.3,7.25
2,1,2017-01-09 11:38:20,2017-01-09 11:42:05,1,1.10,1,N,164,161,1,5.5,0.0,0.5,1.00,0.0,0.3,7.30
3,1,2017-01-09 11:52:13,2017-01-09 11:57:36,1,1.10,1,N,236,75,1,6.0,0.0,0.5,1.70,0.0,0.3,8.50
4,2,2017-01-01 00:00:00,2017-01-01 00:00:00,1,0.02,2,N,249,234,2,52.0,0.0,0.5,0.00,0.0,0.3,52.80


In [6]:
%time dd_to_pd_frame = dd.read_csv(csv_file).compute()

Wall time: 16.2 s


In [9]:
#%time fastparquet.write(csv_file_on+'.parq', pd_frame, compression='SNAPPY')
%time fastparquet.write(csv_file_on+'.parq', pd_frame, compression='GZIP')

Wall time: 56.2 s


In [10]:
!dir

 D 드라이브의 볼륨: d_data
 볼륨 일련 번호: 98F6-9C55

 D:\home\a_dask 디렉터리

2019-02-05  오전 10:17    <DIR>          .
2019-02-05  오전 10:17    <DIR>          ..
2019-02-05  오전 09:54    <DIR>          .ipynb_checkpoints
2019-02-05  오전 05:23            23,743 a01_dask_delayed.ipynb
2019-02-05  오전 09:49    <DIR>          a_dask-dataframe-benchmarking
2019-01-27  오전 07:28             8,874 Dashboard.ipynb
2019-01-27  오전 07:29    <DIR>          dask-worker-space
2019-02-05  오전 04:31            15,292 dask.dataframe_01.ipynb
2019-02-05  오전 04:50             9,646 dask.dataframe_03.ipynb
2019-02-02  오전 10:43             3,061 dask_ml.preprocessing_01.ipynb
2019-02-05  오전 03:41             3,652 Distributed_random_forests.ipynb
2019-02-05  오전 05:08            73,494 map_partitions_01.ipynb
2019-01-29  오후 10:30             6,007 memory_usage.ipynb
2019-02-05  오전 05:21            15,939 mydask.png
2019-02-05  오전 10:12    <DIR>          nyc-taxi-limo-yellow-tripdata-january-2017
2019-02-05  오전 10:11       153,

In [15]:
%%time
# dask parquet
dd_to_pd_frame.to_parquet(csv_file_on+'_dask.parq', compression='GZIP')

Wall time: 35.8 s


In [16]:
%time parq_to_pd = fastparquet.ParquetFile(csv_file_on+'.parq').to_pandas()

Wall time: 14.8 s


In [17]:
%time dd_parq_to_dd_frame = dd.read_parquet(csv_file_on+'.parq').compute()

Wall time: 42 s


In [46]:
%time dd_parq_to_dd_frame = dd.read_parquet(csv_file_on+'.parq', engine='fastparquet').compute()

Wall time: 16.2 s


In [18]:
type(dd_parq_to_dd_frame)

pandas.core.frame.DataFrame

In [20]:
%time pd_frame.to_hdf(csv_file_on+'.h5','tripdata')

Wall time: 54.5 s


In [22]:
%time hdf_to_pd_frame = pd.read_hdf(csv_file_on+'.h5','tripdata')

Wall time: 9.73 s


In [25]:
# del hdf_to_pd_frame
# del dd_parq_to_dd_frame

In [45]:
#dd.read_hdf('myfile.1.hdf5', '/x')  
# dd_to_pd_frame = dd.read_hdf(pattern='yellow_tripdata_2017.1.h5', key='/x') #.compute()
dd_to_pd_frame = dd.read_hdf('yellow_tripdata_2017.1.h5','/df') #.compute()

ValueError: No objects to concatenate